# Gen-Toolbox data analyis notebook

This notebook is part of the gen-toolbox project, a comprehensive tool designed for collating large numbers of VCF files from unique samples, annotating variants, and creating variant frequency tables. The project is tailored to run on substantial servers and has been spearheaded by the Tartu University Hospital Centre of Medical Genetics and the Tartu University Institute of Clinical Medicine, with the backing of the Estonian Research Council grant PSG774.

## Notebook Objective:

This particular notebook delves into the statistical analysis of Single Nucleotide Variants (SNVs) in genomic data. The aim is to:

- Load and process genomic data.
- Compute statistical metrics and ratios.
- Visualize results for a better understanding of the data.

## Prerequisites:

1. **Data Preparation**: Ensure you have the necessary VCF files and gene configurations. The notebook expects CSV or TSV formatted files.

2. **Environment Setup**: If using Docker, ensure Docker is installed and running. Alternatively, ensure you have a Python environment set up with all necessary libraries.

3. **Configuration**: Adjust path variables in the notebook to match the location of your data files.


### 1. Input variables

#### 1.1 Path variables - change these for specifing correct paths

In [ ]:
gene_config = 'gene_config.json' 
args_case =  # add here TSV or CSV file with case data
args_control = # add here TSV or CSV file with control data

#### 1.2 Configuration variables

In [ ]:
is_csv = True # if False, then tsv, if True, then csv
iterations = 100 # number of iterations for permutation test
combination_length=5 # number of genes in a set
case_genes_length = combination_length  # e.g. sets of 5 genes
case_count = extract_number_from_filename(args_case) # extract number from filename
control_count = extract_number_from_filename(args_control) # extract number from filename

## 2. Utility functions

In [ ]:
def load_gene_config(json_file):
    """Load gene configuration from a JSON file."""
    with open(json_file, "r") as file:
        config = json.load(file)
    return config

In [ ]:
def extract_number_from_filename(filename: str) -> int:
    """Extract the number from the given filename."""
    match = re.search(r"\d+", filename)
    return int(match.group()) if match else None

## 3. Data Loading

### 3.1 Loading Gene config

In [ ]:
config = load_gene_config(gene_config)

In [ ]:
# keys in gene_config would show information present in it
config.keys()

In [ ]:
# Lets Show some of itersect genes
config['intersect_genes_tso'][:5]

In [ ]:
# How many intersect genes are there
len(config['intersect_genes_tso'])

In [ ]:
# printing number of genes in each conf
for i in config:
    print(i,':', len(config[i]))

### 3.2 Getting rv_genes and neg_control_genes for use in our test

In [ ]:
rv_genes = config["rv_genes"]
neg_control_genes = config["neg_control_genes"]

### 3.3 Loading case and control SNV data

For csv data file intersect_genes_tso is loaded while for tsv intersect_genes_tshc is loaded. Read the docs for clarification

In [ ]:
if is_csv:
    intersect_genes = config["intersect_genes_tso"]
    df_case = pd.read_csv(args_case, sep=",", header=0)
    df_control = pd.read_csv(args_control, sep=",", header=0)
else:
    intersect_genes = config["intersect_genes_tshc"]
    df_case = pd.read_table(args_case, sep="\t", header=0)
    df_control = pd.read_table(args_control, sep="\t", header=0)

As we have loaded single file for case and control, both dataframes will be same

In [ ]:
df_case.head(2)

In [ ]:
df_control.head(2)

In [ ]:
# Dataframe length
# Each row contains a gene
f'number of genes in case: {len(df_case)}', f'number of genes in control: {len(df_control)}'

### 3.4 Assigning the 1st column the name of gene

In [ ]:
df_case.rename(columns={df_case.columns[0]: "gene"}, inplace = True)
df_control.rename(columns={df_control.columns[0]: "gene"}, inplace = True)

In [ ]:
df_case.columns

### 3.5 Getting list of all genes

In [ ]:
all_genes = df_case['gene']

## 4. Cleaning data

### 4.1 Selecting intersect gene from config

In [ ]:
# Selecting only itersecting genes from config
if intersect_genes is not None:
    # Filter out empty genes and keep only the intersecting genes in both dataframes
    df_case = df_case.dropna(subset=["gene"])
    df_case = df_case[df_case.gene.isin(intersect_genes)]
    df_control = df_control.dropna(subset=["gene"])
    df_control = df_control[df_control.gene.isin(intersect_genes)]
    df_case.reset_index(drop=True, inplace=True)
    df_control.reset_index(drop=True, inplace=True)

In [ ]:
# Dataframe length after selecting only intersecting genes available in config
f'number of genes in case: {len(df_case)}', f'number of genes in control: {len(df_control)}'

In [ ]:
#Check if dataframes are of equal length
if len(df_case.index) != len(df_control.index):
    print("WARNING: Case dataframe length does not match control dataframe length! The intersect of both dataframes will be analysed.")

### 4.2 Taking intersect of both data frames and sorting on gene column

This step is performed to ensure that same genes for case and control are present and we can apply indexwise operation in our statistical test. Randomly selected indices will produce same genes from both data frames.

Sorting is done because if we select gene names randomly and then filter both dataframes for selected gene names then it will be a slow process


In [ ]:
# Take only the intersect of two dataframes based on gene column
intersection_values = set(df_case['gene']).intersection(df_control['gene'])
df_case = df_case[df_case["gene"].isin(intersection_values)]
df_case = df_case.sort_values(by="gene")
df_control = df_control[df_control["gene"].isin(intersection_values)]
df_control = df_control.sort_values(by="gene")
df_case.reset_index(drop=True, inplace=True)
df_control.reset_index(drop=True, inplace=True)

In [ ]:
# The rows must match in order to do index based math
assert df_case["gene"].equals(df_control["gene"]), "Case and control dataframe indices do not match!"

## 5. Statistical test

### 5.1 Initializing empty dataframes to store results

In [ ]:
fraction_results_1 = pd.DataFrame()
fraction_results_2 = pd.DataFrame(columns=df_case.columns[1:].tolist())

### 5.2 Calculating desired variables

If we have passed only 1 file it will make expected_ratio to be 1

In [ ]:
expected_ratio = case_count / control_count
df_case.reset_index(drop=True, inplace=True)
df_control.reset_index(drop=True, inplace=True)
columns_to_add = df_case.columns[1:]

# Calculate the mean of the case and control dataframes
df_case_mean = df_case.mean()
df_control_mean = df_control.mean()

num_columns = df_case.shape[1]

print("Case means \n{0}\nControl means \n{1}".format(df_case_mean, df_control_mean))
print("Expected ratio cases / controls: {0}, log2 {1}".format(expected_ratio, np.log2(expected_ratio)))
print("Expected ratio cases / controls by group (log2): \n {0}".format(np.log2(df_case_mean / df_control_mean)))

In [ ]:
# Divisible columns, look at the ratios grossly
divison_result_gross = df_case[columns_to_add] / df_control[columns_to_add]
r = divison_result_gross[divison_result_gross[:-1] > expected_ratio].dropna(how="all")
r["gene"] = df_case["gene"]  ## Do nothing

### 5.2 Random selection of indices for statistical test

In [ ]:
indices = np.array([np.random.choice(df_case.index, size=case_genes_length, replace=False) for _ in range(iterations)])

### 5.3 Taking sum for columns against selected indices for each iteration

In [ ]:
# Calculate the sum of total variants for the case and control groups using the sampled indices
total_variants_case = np.array([df_case.iloc[idx, 1:].sum().to_numpy() for idx in indices])
total_variants_control = np.array([df_control.iloc[idx, 1:].sum().to_numpy() for idx in indices])

In [ ]:
total_variants_case[0]

### 5.4 Ratios vector for each column based on given condition to include sum

In [ ]:
ratios_vector = np.where(np.logical_and(total_variants_case > 1, total_variants_control > 1),\
                total_variants_case / total_variants_control,\
                np.NaN)

In [ ]:
# numer of iterations and column number can be seen in shape
ratios_vector.shape

In [ ]:
# Where sum is less than 1 NaN will be placed for the ratio in that
ratios_vector[0]

### 5.5 Satisfying below statistical condition to get results




1. $$ \log_2\left(\frac{\text{ratios\_vector}}{\text{expected\_ratio}}\right) > 0.7 $$


2. $$ \text{total\_variants\_case} > \text{case\_genes\_length} $$


3. $$ \text{total\_variants\_control} > \text{case\_genes\_length} $$


In [ ]:
fraction_results_tmp = pd.DataFrame(ratios_vector, columns=df_case.columns[1:].tolist())

complex_condition = np.logical_and(
                    np.log2(ratios_vector/expected_ratio) > 0.7,
                    total_variants_case > case_genes_length,
                    total_variants_control > case_genes_length
                    ).T.flatten()

gene = np.array([df_case.gene[idx].to_numpy() for idx in indices])
case = np.array([df_case.iloc[idx, 1:].to_numpy().T for idx in indices]).transpose((1, 0, 2))
control = np.array([df_control.iloc[idx, 1:].to_numpy().T for idx in indices]).transpose((1, 0, 2))
case = case.reshape((-1, case.shape[-1]))
control = control.reshape((-1, control.shape[-1]))
case_control = case/control

fraction_results_1 = pd.DataFrame()
fraction_results_1['frequency_bin'] = np.repeat(fraction_results_tmp.columns, len(fraction_results_tmp))
fraction_results_1['burden_ratio'] = fraction_results_tmp.to_numpy().T.flatten()
fraction_results_1['gene'] = list(np.tile(gene, (len(df_case.columns[1:].tolist()),1)))
fraction_results_1['case'] = list(case)
fraction_results_1['control'] = list(control)
fraction_results_1['case_control'] = list(case_control)
fraction_results_1 = fraction_results_1[complex_condition].reset_index(drop=True)
fraction_results_1['burden_event'] = list(range(len(fraction_results_1)))

In [ ]:
# while working with case and control data this condition will get satisfied for some samples
fraction_results_1

In [ ]:
# Verify True and False condition indexes
print(list(complex_condition))

### 5.6 Storing Dataframe

In [ ]:
fraction_results_1.to_csv("{0}_{1}_{2}.csv".format(case_count, control_count, iterations))

### 5.7 Getting ratio with expected ratio

In [ ]:
fraction_results_2 = pd.DataFrame(ratios_vector/expected_ratio, columns=df_case.columns[1:].tolist())

### 5.8 Preparing for plot

In [ ]:
fig, axs = plt.subplots(
    len(fraction_results_2.columns),
    1,
    sharex="none",
    tight_layout=False,
    figsize=(12, 24),
)
i = 0
for frequency_column in fraction_results_2.columns:
    fraction_results_2[frequency_column].dropna(inplace=True)

    if fraction_results_2[frequency_column].any():
        # Average sample normalization enrichment ratios for "likely impactful" and "likely non-impactful" genes
        q_avg = np.divide(
            np.sum(df_case[df_case.gene.isin(rv_genes)][frequency_column]),
            np.sum(df_control[df_control.gene.isin(rv_genes)][frequency_column]),)
        q_avg_control_group = np.divide(
            np.sum(df_case[df_case.gene.isin(neg_control_genes)][frequency_column]),
            np.sum(
                df_control[df_control.gene.isin(neg_control_genes)][frequency_column]),)
        print(
            "Impact group (av-norm. ): {0}, case_genes_enrichment: {1}, control_genes_enrichment: {2}".format(
                frequency_column, q_avg, q_avg_control_group
            )
        )
        fraction_results_2[frequency_column + "_log2"] = np.log2(
            fraction_results_2[frequency_column]).dropna()

        # Unused block
        mu, std = sp.norm.fit(fraction_results_2[frequency_column + "_log2"].dropna())
        percentile_99 = np.percentile(fraction_results_2[frequency_column + "_log2"].dropna(), 99)
        xmin, xmax = (
            fraction_results_2[frequency_column + "_log2"].min(),
            fraction_results_2[frequency_column + "_log2"].dropna().max(),
        )
        x = np.linspace(mu - 3 * std, mu + 3*std, 100)
        _, bins, _ = axs[i].hist(
            fraction_results_2[frequency_column+ "_log2"],
            density=False,
            log=False,
            histtype="stepfilled",
            stacked=True,
            bins=500,
        )
        p = sp.norm.pdf(bins, mu, std)
        axs[i].set_title(
            "{0}: mean_enrichment={1} positive_genes={2} negative_genes={3} 99th percentile (purple)={4}".format(
                fraction_results_2[frequency_column].name + "_log2",
                np.round(fraction_results_2[frequency_column + "_log2"].mean(), 2),
                np.round(np.log2(q_avg), 2),
                np.round(np.log2(q_avg_control_group), 2),
                np.round(percentile_99, 2)
                )
        )
        axs[i].axvline(percentile_99, color="purple")
        i +=1
plt.show()
print("Done {0} iterations".format(iterations))


# Library Imports

In [ ]:
import argparse
import datetime
import json
import re
import sys
import uuid
from pathlib import Path

import scipy.stats as sp
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd